In [4]:
import pandas as pd 
import os
import Bio
from Bio import SeqIO
from Bio.SeqUtils import gc_fraction
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
from Bio.Align import MultipleSeqAlignment
from Bio.Align.Applications import ClustalwCommandline
import collections
from Bio import AlignIO
import random
import numpy as np
import sys

#sys.path.append(os.path.abspath('/home/nimar/PLP_directRNA_design_V2/PLP_directRNA_design/'))
#from PLP_directRNA_design import probedesign as plp

In [ ]:
number_of_seqs= 10 # number of regions mapped/PLP
final_designed=5 # number of PLPs designed at the end/gene
plp_length=30
mismatches = 6


# Extcrating common regions

In [1]:
from BCBio import GFF
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
from collections import defaultdict
import argparse

In [30]:



def compute_shared_regions(isoforms):
    """
    Computes the shared region and depth across multiple isoforms.
    """
    if not isoforms:
        return "", []

    ref_seq = isoforms[0]
    seq_length = len(ref_seq)
    depth = [0] * seq_length

    for isoform in isoforms:
        for i in range(min(len(isoform), seq_length)):
            if isoform[i] == ref_seq[i]:
                depth[i] += 1

    shared_seq = "".join(ref_seq[i] if depth[i] == len(isoforms) else "-" for i in range(seq_length))

    return shared_seq, depth

def main(gtf_file, output_file, genes_str=None):
    """
    Main function to parse a GTF file (optionally filtered by a gene list)
    and compute the shared regions for transcripts and CDS.

    Args:
        gtf_file (str): Path to the GTF file.
        output_file (str): Path to the output file.
        genes_str (str): Comma-separated string of gene IDs. If None, all genes are processed.
    """
    # Convert the comma-separated string into a set of gene IDs (if provided)
    if genes_str:
        genes_of_interest = set([g.strip().lower() for g in genes_str.split(",")])
        print(f"Processing genes: {', '.join(genes_of_interest)}")
    else:
        genes_of_interest = None
        print("No gene list provided. Processing all genes.")

    with open(output_file, "w") as out_handle:
        for gene_id, transcripts, cds_regions in parse_gtf_sequential(gtf_file, genes_of_interest):
            # Write transcript-level results
            if transcripts:
                shared_transcript_seq, depth_transcript = compute_shared_regions(transcripts)
                out_handle.write(f">{gene_id}_shared_transcript\n{shared_transcript_seq}\n")
                out_handle.write(f">{gene_id}_depth_transcript\n{' '.join(map(str, depth_transcript))}\n")

            # Write CDS-level results
            if cds_regions:
                shared_cds_seq, depth_cds = compute_shared_regions(cds_regions)
                out_handle.write(f">{gene_id}_shared_CDS\n{shared_cds_seq}\n")
                out_handle.write(f">{gene_id}_depth_CDS\n{' '.join(map(str, depth_cds))}\n")


if __name__ == "__main__":
    parser = argparse.ArgumentParser(description="Extract overlapping transcripts and CDS from a GTF.")
    parser.add_argument("--gtf", required=True, help="Path to the input GTF file.")
    parser.add_argument("--out", required=True, help="Path to the output file.")
    parser.add_argument("--genes", default=None, help="Comma-separated list of gene IDs to filter.")
    args = parser.parse_args()

    main(args.gtf, args.out, args.genes)


usage: ipykernel_launcher.py [-h] --gtf GTF --out OUT [--genes GENES]
ipykernel_launcher.py: error: the following arguments are required: --gtf, --out


SystemExit: 2

/Users/nimra236/anaconda3/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3516: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


# Here is the main functions to use

In [ ]:
from BCBio import GFF
from collections import defaultdict
import argparse

def compute_shared_sequences(isoforms):
    """
    Computes the shared sequence and depth across multiple isoforms.
    """
    if not isoforms:
        return "", []

    ref_seq = isoforms[0]
    seq_length = len(ref_seq)
    depth = [0] * seq_length

    for isoform in isoforms:
        for i in range(min(len(isoform), seq_length)):
            if isoform[i] == ref_seq[i]:
                depth[i] += 1

    shared_seq = "".join(ref_seq[i] if depth[i] == len(isoforms) else "-" for i in range(seq_length))

    return shared_seq, depth

def extract_region(feature, record):
    """
    Extracts the nucleotide sequence of a given feature type (CDS) from the genomic record.
    """
    seq_fragments = []
    print('HERE') 
    for sub_feature in feature.sub_features:

        start, end = sub_feature.location.start, sub_feature.location.end
        seq_fragments.append(str(record.seq[start:end]))

    return "".join(seq_fragments) if seq_fragments else None



    # After processing all records, print the matched genes
    print("\nMatched Genes:")
    for gene in matched_genes:
        print(gene)


parse_gtf_sequential(gtf_file, genes_of_interest, identifier_type='gene_id', gff_src = 'ensembl')
#print(gene_id)



Matched Genes:


In [147]:
import pandas as pd
import numpy as np
import re

def parse_gtf_to_dataframe(gtf_path: str) -> pd.DataFrame:
    """
    Parses a GTF (Gene Transfer Format) file into a structured Pandas DataFrame.

    Args:
        gtf_path (str): Path to the GTF file.

    Returns:
        pd.DataFrame: A DataFrame containing parsed GTF data.
    """
    print('Loading GTF file....')
    # Define column names
    col_names = [
        'seqname', 'source', 'feature', 'start', 'end', 'score', 
        'strand', 'frame', 'attribute'
    ]
    
    # Read the GTF file, skipping comments
    df = pd.read_csv(gtf_path, sep='\t', header=None, names=col_names, comment='#', dtype={'score': str})

    # Define attributes to extract
    attributes = [
        'gene_id', 'transcript_id', 'exon_number', 'gene_name', 'gene_source', 
        'gene_biotype', 'transcript_name', 'transcript_source', 'transcript_biotype', 
        'protein_id', 'exon_id', 'tag'
    ]

    # Create regex patterns for attribute extraction
    attr_patterns = {attr: rf'{attr} "([^"]*)"' for attr in attributes}

    # Extract attributes using vectorized operations
    for attr, pattern in attr_patterns.items():
        df[attr] = df['attribute'].str.extract(pattern)

    # Drop the original attribute column
    df.drop(columns=['attribute'], inplace=True)

    return df


def parse_gtf(gtf_file, genes_of_interest=None, identifier_type='gene_id', gene_feature='CDS'):
    """
    Parses a GTF file and yields data only for the specified genes_of_interest.

    Args:
        gtf_file (str): Path to the GTF file.
        genes_of_interest (set or None): A set of gene IDs or names to parse.
                                         If None, parse all genes in the GTF.
        identifier_type (str): Type of identifier provided ('gene_id' or 'gene_name').

    Returns:
        pd.DataFrame: A DataFrame containing parsed GTF data.
    """
    print('Parsing GTF file....')
    # Read the GTF file into a DataFrame
    gtf_df = parse_gtf_to_dataframe(gtf_file)

    # Check if a gene list is provided and filter accordingly
    if genes_of_interest:
        if identifier_type == 'gene_id':
            gtf_df = gtf_df[gtf_df['gene_id'].str.lower().isin(genes_of_interest) & (gtf_df['feature'] == gene_feature)]
        elif identifier_type == 'gene_name':
            gtf_df = gtf_df[gtf_df['gene_name'].str.lower().isin(genes_of_interest)& (gtf_df['feature'] == gene_feature)]
        else:
            raise ValueError("identifier_type must be 'gene_id' or 'gene_name'")

    return gtf_df


def merge_regions_and_coverage(genes_of_interest, gtf_df):
    """
    Merge CDS regions and computes average coverage for the specified genes.

    Args:
        genes_of_interest (set): A set of gene names to process.
        gtf_df (pd.DataFrame): A DataFrame containing parsed GTF data.

    Returns:
        pd.DataFrame: A DataFrame containing merged CDS regions and average coverage.
    """
    print('Merge regions and calculating coverage....')
    # Initialize an empty list to store results
    merged_regions = []

    for gn in genes_of_interest:
        # Subset the DataFrame for the given gene_name
        isoforms = gtf_df[gtf_df['gene_name'].str.lower() == gn].copy()
        
        # Sort by chromosome, strand, and start position
        isoforms = isoforms.sort_values(by=['seqname', 'strand', 'start'])

        # Determine the full genomic range
        min_start = isoforms['start'].min()
        max_end = isoforms['end'].max()

        # Create a NumPy array to track coverage over this genomic range
        coverage_array = np.zeros(max_end - min_start + 1, dtype=int)

        # Dictionary to track merged CDS regions
        merged = []
        
        for _, row in isoforms.iterrows():
            if not merged:
                merged.append(row.to_dict())  # First entry, add as-is
            else:
                prev = merged[-1]

                if row['start'] <= prev['end']:  # Overlapping region
                    merged[-1]['end'] = max(prev['end'], row['end'])  # Expand region
                    merged[-1]['transcript_id'] += ";" + row['transcript_id']  # Append transcript_id
                else:
                    merged.append(row.to_dict())  # Start new region

            # Update coverage using NumPy slicing (avoids looping over each position)
            coverage_array[row['start'] - min_start : row['end'] - min_start + 1] += 1

        # Convert merged results to a DataFrame
        merged_df = pd.DataFrame(merged)

        # Compute the average coverage for each merged region efficiently
        coverage_values = []
        for _, region in merged_df.iterrows():
            region_slice = coverage_array[region['start'] - min_start : region['end'] - min_start + 1]
            avg_coverage = np.mean(region_slice)  # Vectorized mean calculation
            coverage_values.append(avg_coverage)

        # Add coverage column to DataFrame
        merged_df['coverage'] = coverage_values

        merged_regions.append(merged_df)

    # Concatenate all gene-specific DataFrames into a final result
    final_df = pd.concat(merged_regions, ignore_index=True)

    # Keep only relevant columns
    final_df = final_df[['seqname', 'start', 'end', 'strand', 'gene_name', 'transcript_id', 'coverage']]

    # Display the DataFrame with coverage
    return(final_df)


def main(gtf_file, output_file, genes_str=None, identifier_type='gene_id', gene_feature='CDS'):
    """
    Main function to parse a GTF file (optionally filtered by a gene list)
    and print matched gene information, also merge regions and calculate coverage.

    Args:
        gtf_file (str): Path to the GTF file.
        output_file (str): Path to the output text file.
        genes_str (str): Comma-separated string of gene IDs or names to filter.
        identifier_type (str): Type of identifier provided ('gene_id' or 'gene_name').
        gene_feature (str): Feature type to extract (default = 'CDS').
    """
    # Convert the comma-separated string into a set of gene IDs or names (if provided)
    if genes_str:
        genes_of_interest = set([g.strip().lower() for g in genes_str.split(",")])
        print(f"Processing genes: {', '.join(genes_of_interest)}")
    else:
        genes_of_interest = None
        print("No gene list provided. Processing all genes.")

    # Parse the GTF file and filter by gene list
    gtf_df = parse_gtf(gtf_file, genes_of_interest, identifier_type, gene_feature)

    # Merge regions and calculate coverage
    merged_cov_df = merge_regions_and_coverage(genes_of_interest, gtf_df)

    # Write the merged results to an output file
    merged_cov_df.to_csv(output_file, sep='\t', index=False)

    print(f"Results saved to {output_file}")

if __name__ == "main":
    parser = argparse.ArgumaentParser(description ="Parse GTF file and calculate coverage")
    parser.add_argument("--gtf_file", required=True, help="Path to the GTF file")
    parser.add_argument("--output_file", required=True, help="Path to the output file")
    parser.add_argument("--genes_str", default=None, help="Comma-separated list of gene IDs or names to filter")
    parser.add_argument("--identifier_type", default='gene_id', help="Type of identifier provided ('gene_id' or 'gene_name')")
    parser.add_argument("--gene_feature", default='CDS', help="Feature type to extract (default = 'CDS')")

gtf_file='/Users/nimra236/Dropbox/NBIS/Projects/PLP_directRNA_design_V2/data/tmp.gtf'
genes_str='gRik2, prlh'
output_file='matched_genes.txt'
genes_of_interest = set([g.strip().lower() for g in genes_str.split(",")])
gtf_df = parse_gtf(gtf_file, genes_of_interest, identifier_type='gene_name')
merged_cov_df = merge_regions_and_coverage(genes_of_interest, gtf_df)
merged_cov_df.to_csv(output_file, sep='\t', index=False)

Parsing GTF file....
Loading GTF file....
Merge regions and calculating coverage....


In [128]:
gtf_df

,seqname,source,feature,start,end,score,strand,frame,gene_id,transcript_id,exon_number,gene_name,gene_source,gene_biotype,transcript_name,transcript_source,transcript_biotype,protein_id,exon_id,tag
4,10,havana,CDS,49659241,49659355,.,-,0,ENSMUSG00000056073,ENSMUST00000218823,2,Grik2,ensembl_havana,protein_coding,Grik2-207,havana,protein_coding,ENSMUSP00000151921,NaN,CCDS
7,10,havana,CDS,49454231,49454398,.,-,2,ENSMUSG00000056073,ENSMUST00000218823,3,Grik2,ensembl_havana,protein_coding,Grik2-207,havana,protein_coding,ENSMUSP00000151921,NaN,CCDS
9,10,havana,CDS,49449660,49449917,.,-,2,ENSMUSG00000056073,ENSMUST00000218823,4,Grik2,ensembl_havana,protein_coding,Grik2-207,havana,protein_coding,ENSMUSP00000151921,NaN,CCDS
11,10,havana,CDS,49411414,49411595,.,-,2,ENSMUSG00000056073,ENSMUST00000218823,5,Grik2,ensembl_havana,protein_coding,Grik2-207,havana,protein_coding,ENSMUSP00000151921,NaN,CCDS
13,10,havana,CDS,49404083,49404136,.,-,0,ENSMUSG00000056073,ENSMUST00000218823,6,Grik2,ensembl_havana,protein_coding,Grik2-207,havana,protein_coding,ENSMUSP00000151921,NaN,CCDS
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
252,10,ensembl,CDS,49008775,49009000,.,-,0,ENSMUSG00000056073,ENSMUST00000105484,14,Grik2,ensembl_havana,protein_coding,Grik2-202,ensembl,protein_coding,ENSMUSP00000101124,NaN,CCDS
254,10,ensembl,CDS,48989466,48989716,.,-,2,ENSMUSG00000056073,ENSMUST00000105484,15,Grik2,ensembl_havana,protein_coding,Grik2-202,ensembl,protein_coding,ENSMUSP00000101124,NaN,CCDS
256,10,ensembl,CDS,48977207,48977368,.,-,0,ENSMUSG00000056073,ENSMUST00000105484,16,Grik2,ensembl_havana,protein_coding,Grik2-202,ensembl,protein_coding,ENSMUSP00000101124,NaN,CCDS
262,1,ensembl_havana,CDS,90880830,90880926,.,+,0,ENSMUSG00000090550,ENSMUST00000166281,1,Prlh,ensembl_havana,protein_coding,Prlh-201,ensembl_havana,protein_coding,ENSMUSP00000130768,NaN,CCDS


In [ ]:
def main(gtf_file, output_file, genes_str=None, identifier_type='gene_id'):
    """
    Main function to parse a GTF file (optionally filtered by a gene list)
    and print matched gene information.

    Args:
        gtf_file (str): Path to the GTF file.
        output_file (str): Path to the output text file.
        genes_str (str): Comma-separated string of gene IDs or names to filter.
        identifier_type (str): Type of identifier provided ('gene_id' or 'gene_name').
    """
    # Convert the comma-separated string into a set of gene IDs/names (lowercase for case-insensitivity)
    if genes_str:
        genes_of_interest = set([g.strip().lower() for g in genes_str.split(",")])
    else:
        genes_of_interest = None

    with open(output_file, "w") as out_handle:
        for gene_id, gene_name, transcripts, cds_sequences in parse_gtf_sequential(gtf_file, genes_of_interest, identifier_type):
            # Print matched gene information
            print(f"Gene ID: {gene_id}")
            print(f"Gene Name: {gene_name}")

            # Optionally, write to the output file if needed
            out_handle.write(f">Gene ID: {gene_id}\n")
            out_handle.write(f"Gene Name: {gene_name}\n")

            # Print transcript features
            if transcripts:
                print("Transcript Features:")
                for transcript in transcripts:
                    print(f"  - Location: {transcript.location}")
                    # You can print more details if needed
                    out_handle.write(f"  Transcript Location: {transcript.location}\n")

            # Print CDS features
            if cds_sequences:
                print("CDS Features:")
                for cds in cds_sequences:
                    print(f"  - Location: {cds.location}")
                    # You can print more details if needed
                    out_handle.write(f"  CDS Location: {cds.location}\n")


In [19]:
import plp_directrna_design as plp
gtf_file = '../data/tmp.gtf'
genes_str = ['grik2', 'prlh']
gtf_df, genes_of_interest = plp.parse_gtf(gtf_file, genes_str, 'gene_name', 'CDS')

# Merge regions and calculate coverage
merged_cov_df = plp.merge_regions_and_coverage(genes_of_interest, gtf_df)
merged_cov_df


Parsing GTF file....
Loading GTF file....
There are 1352 queries in the gtff file


AttributeError: 'list' object has no attribute 'split'

In [15]:
gtf_df
genes_of_interest = set(['grik2'])
print(gtf_df['feature'].unique())

gtf_df[gtf_df['gene_name'].str.lower().isin(genes_of_interest) & (gtf_df['feature'] == 'CDS')]



['gene' 'transcript' 'exon' 'CDS' 'start_codon' 'stop_codon'
 'five_prime_utr' 'three_prime_utr']


,seqname,source,feature,start,end,score,strand,frame,gene_id,transcript_id,exon_number,gene_name,gene_source,gene_biotype,transcript_name,transcript_source,transcript_biotype,protein_id,exon_id,tag
4,10,havana,CDS,49659241,49659355,.,-,0,ENSMUSG00000056073,ENSMUST00000218823,2,Grik2,ensembl_havana,protein_coding,Grik2-207,havana,protein_coding,ENSMUSP00000151921,NaN,CCDS
7,10,havana,CDS,49454231,49454398,.,-,2,ENSMUSG00000056073,ENSMUST00000218823,3,Grik2,ensembl_havana,protein_coding,Grik2-207,havana,protein_coding,ENSMUSP00000151921,NaN,CCDS
9,10,havana,CDS,49449660,49449917,.,-,2,ENSMUSG00000056073,ENSMUST00000218823,4,Grik2,ensembl_havana,protein_coding,Grik2-207,havana,protein_coding,ENSMUSP00000151921,NaN,CCDS
11,10,havana,CDS,49411414,49411595,.,-,2,ENSMUSG00000056073,ENSMUST00000218823,5,Grik2,ensembl_havana,protein_coding,Grik2-207,havana,protein_coding,ENSMUSP00000151921,NaN,CCDS
13,10,havana,CDS,49404083,49404136,.,-,0,ENSMUSG00000056073,ENSMUST00000218823,6,Grik2,ensembl_havana,protein_coding,Grik2-207,havana,protein_coding,ENSMUSP00000151921,NaN,CCDS
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
248,10,ensembl,CDS,49120421,49120539,.,-,1,ENSMUSG00000056073,ENSMUST00000105484,12,Grik2,ensembl_havana,protein_coding,Grik2-202,ensembl,protein_coding,ENSMUSP00000101124,NaN,CCDS
250,10,ensembl,CDS,49116675,49116892,.,-,2,ENSMUSG00000056073,ENSMUST00000105484,13,Grik2,ensembl_havana,protein_coding,Grik2-202,ensembl,protein_coding,ENSMUSP00000101124,NaN,CCDS
252,10,ensembl,CDS,49008775,49009000,.,-,0,ENSMUSG00000056073,ENSMUST00000105484,14,Grik2,ensembl_havana,protein_coding,Grik2-202,ensembl,protein_coding,ENSMUSP00000101124,NaN,CCDS
254,10,ensembl,CDS,48989466,48989716,.,-,2,ENSMUSG00000056073,ENSMUST00000105484,15,Grik2,ensembl_havana,protein_coding,Grik2-202,ensembl,protein_coding,ENSMUSP00000101124,NaN,CCDS
